In [3]:
import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.metrics import mean_squared_error

import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'pandas'

In [2]:
import pickle

In [3]:
import mlflow


mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

2026/02/07 04:56:30 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2026/02/07 04:56:30 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.


<Experiment: artifact_location='/workspaces/mlops-demo/02-experiment-tracking/mlruns/2', creation_time=1770438136170, experiment_id='2', last_update_time=1770438136170, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [4]:
df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')

In [5]:
df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df['duration'] = df.duration.dt.total_seconds() / 60

In [6]:
df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

In [7]:
categorical = ['PULocationID', 'DOLocationID']

In [8]:
df[categorical] = df[categorical].astype(str)

In [9]:
train_dicts = df[categorical].to_dict(orient='records')

In [10]:
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

In [11]:
target = 'duration'
y_train = df[target].values

In [12]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].astype('str')
    
    return df

In [13]:
df_val = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

In [14]:
val_dicts = df_val[categorical].to_dict(orient='records')

In [15]:
X_val = dv.transform(val_dicts) 
y_val = df_val.duration.values

In [ ]:
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope
import mlflow

train = xgb.DMatrix(X_train[:100], label=y_train[100])
valid = xgb.DMatrix(X_val, label=y_val)

def objective(params):
    # Convert params to proper types
    params_converted = {
        'max_depth': int(params['max_depth']),
        'learning_rate': params['learning_rate'],
        'reg_alpha': params['reg_alpha'],
        'reg_lambda': params['reg_lambda'],
        'min_child_weight': params['min_child_weight'],
        'objective': 'reg:squarederror',  # Updated from deprecated 'reg:linear'
        'seed': 42
    }
    
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params_converted)
        
        booster = xgb.train(
            params=params_converted,
            dtrain=train,
            num_boost_round=10,
            evals=[(valid, 'validation')],
            early_stopping_rounds=10,
            verbose_eval=False  # Suppress output for speed
        )
        
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)
        
    return {'loss': rmse, 'status': STATUS_OK}

search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=20,
    trials=Trials()
)

  0%|          | 0/50 [00:45<?, ?trial/s, best loss=?]


KeyboardInterrupt: 